In [5]:
pip install lightgbm

  Using cached lightgbm-4.6.0-py3-none-win_amd64.whl.metadata (17 kB)
Using cached lightgbm-4.6.0-py3-none-win_amd64.whl (1.5 MB)
Note: you may need to restart the kernel to use updated packages.


In [45]:
import numpy as np
import pandas as pd
import lightgbm as gbm

In [47]:
dataset = pd.read_csv("sc_data.csv")

In [49]:
target_col = "Number of products sold"
features = ['Price','Availability', 'Stock levels', 'Lead times', 'Order quantities']

In [51]:
num_folds = 5

In [53]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = num_folds, shuffle = True, random_state = 42)

In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset[features], dataset[target_col], test_size = 0.2, random_state = 42)

In [57]:
params= {
    'objective': 'regression',
    'boostin_type': 'gbdt',
    'metric':'mse',
    'learning_rate': 0.05,
    'num_leaves': 31,
    'feature_fraction': 0.9
}

In [61]:
for train_idex, test_idex in kf.split(dataset):
    train_data = dataset.loc[train_idex,features]
    train_target = dataset.loc[train_idex,target_col]
    test_data = dataset.loc[test_idex, features]
    test_target = dataset.loc[test_idex, target_col]

In [63]:
train_data = gbm.Dataset(X_train, label = y_train)

In [65]:
num_round = 100
best = gbm.train(params, train_data, num_round)

[LightGBM] [Warning] Unknown parameter: boostin_type
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: boostin_type
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000104 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 124
[LightGBM] [Info] Number of data points in the train set: 80, number of used features: 5
[LightGBM] [Info] Start training from score 453.200000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [67]:
y_pred = best.predict(X_test, num_iteration = best.best_iteration)

In [69]:
y_pred

array([602.26477999, 462.50809993, 470.25702294, 382.77090386,
       325.46704061, 413.25164768, 375.62754257, 557.18903774,
       332.66809798, 444.34647958, 346.02050077, 409.1354491 ,
       497.41937115, 360.4174601 , 336.97107195, 346.89585609,
       442.85618876, 354.04827904, 570.12310461, 269.46575199])

In [71]:
from sklearn.metrics import mean_squared_error

In [75]:
mse_scores = []

mse = mean_squared_error(test_target, y_pred)
mse_scores.append(mse)

In [79]:
sum(mse_scores)/num_folds

13258.838709275815